In [1]:
import os
import numpy as np
import pandas as pd
import pickle
from pycoingecko import CoinGeckoAPI
import quandl
import datetime
import statsmodels.api as sm
from scipy import stats
import pickle
import requests



import time
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns; sns.set()

import sys
from PIL import Image # for logos
import io
from itertools import product
import warnings

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid1.inset_locator import mark_inset

from plotly import tools
import plotly.offline as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
py.init_notebook_mode(connected=True)



In [2]:

df = pd.read_csv("/Users/benjaminzelaya/Desktop/PI_DA-main/PI_2_Henry/cripto_data_sorted.csv")
df

,id,symbol,name,current_price,market_cap,market_cap_rank,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,ath,ath_change_percentage,ath_date
0,bitcoin,btc,Bitcoin,"29,079.00","566,002,645,211.00",1,"29,408.00","29,053.00",-254.49,-86.76%,"-4,949,362,744.32",-86.69%,"69,045.00",-57.87%,2021-11-10T14:24:11.849Z
1,ethereum,eth,Ethereum,"1,821.40","218,868,994,489.00",2,"1,842.75","1,818.53",-16.86,-91.73%,"-2,053,000,983.60",-92.93%,"4,878.26",-62.65%,2021-11-10T14:24:19.604Z
2,tether,usdt,Tether,1.00,"83,135,475,584.00",3,1.00,1.00,0.00,26.67%,"-208,350,642.64",-25.00%,1.32,-24.40%,2018-07-24T00:00:00.000Z
3,binancecoin,bnb,BNB,233.03,"35,855,531,599.00",4,239.20,232.73,-5.55,-232.76%,"-875,347,235.56",-238.31%,686.31,-66.06%,2021-05-10T07:24:17.097Z
4,ripple,xrp,XRP,0.59,"31,317,407,561.00",5,0.63,0.59,-0.03,-495.79%,"-1,652,308,724.09",-501.16%,3.40,-82.57%,2018-01-07T00:00:00.000Z
5,usd-coin,usdc,USD Coin,1.00,"26,016,993,917.00",6,1.00,1.00,0.00,11.30%,"74,236,489.00",28.62%,1.17,-14.62%,2019-05-08T00:40:28.300Z
6,dogecoin,doge,Dogecoin,0.07,"9,725,167,619.00",8,0.07,0.07,-0.00,-614.68%,"-629,467,308.19",-607.91%,0.73,-90.57%,2021-05-08T05:08:23.458Z
7,cardano,ada,Cardano,0.27,"9,563,403,410.00",9,0.29,0.27,-0.02,-538.16%,"-548,125,916.29",-542.08%,3.09,-91.19%,2021-09-02T06:00:10.474Z
8,solana,sol,Solana,23.13,"9,391,870,633.00",10,24.92,22.99,-1.65,-666.59%,"-672,448,219.60",-668.15%,259.96,-91.13%,2021-11-06T21:54:35.825Z
9,polkadot,dot,Polkadot,4.73,"5,980,946,332.00",12,4.96,4.71,-0.22,-447.38%,"-279,562,748.71",-446.55%,54.98,-91.43%,2021-11-04T14:10:09.301Z


In [3]:
df.isnull().sum().sum()
df.isna().sum()

id                                  0
symbol                              0
name                                0
current_price                       0
market_cap                          0
market_cap_rank                     0
high_24h                            0
low_24h                             0
price_change_24h                    0
price_change_percentage_24h         0
market_cap_change_24h               0
market_cap_change_percentage_24h    0
ath                                 0
ath_change_percentage               0
ath_date                            0
dtype: int64

In [4]:
df['ath_date'] = pd.to_datetime(df['ath_date'])
df['ath_date'] = df['ath_date'].dt.date

In [5]:
df

,id,symbol,name,current_price,market_cap,market_cap_rank,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,ath,ath_change_percentage,ath_date
0,bitcoin,btc,Bitcoin,"29,079.00","566,002,645,211.00",1,"29,408.00","29,053.00",-254.49,-86.76%,"-4,949,362,744.32",-86.69%,"69,045.00",-57.87%,2021-11-10
1,ethereum,eth,Ethereum,"1,821.40","218,868,994,489.00",2,"1,842.75","1,818.53",-16.86,-91.73%,"-2,053,000,983.60",-92.93%,"4,878.26",-62.65%,2021-11-10
2,tether,usdt,Tether,1.00,"83,135,475,584.00",3,1.00,1.00,0.00,26.67%,"-208,350,642.64",-25.00%,1.32,-24.40%,2018-07-24
3,binancecoin,bnb,BNB,233.03,"35,855,531,599.00",4,239.20,232.73,-5.55,-232.76%,"-875,347,235.56",-238.31%,686.31,-66.06%,2021-05-10
4,ripple,xrp,XRP,0.59,"31,317,407,561.00",5,0.63,0.59,-0.03,-495.79%,"-1,652,308,724.09",-501.16%,3.40,-82.57%,2018-01-07
5,usd-coin,usdc,USD Coin,1.00,"26,016,993,917.00",6,1.00,1.00,0.00,11.30%,"74,236,489.00",28.62%,1.17,-14.62%,2019-05-08
6,dogecoin,doge,Dogecoin,0.07,"9,725,167,619.00",8,0.07,0.07,-0.00,-614.68%,"-629,467,308.19",-607.91%,0.73,-90.57%,2021-05-08
7,cardano,ada,Cardano,0.27,"9,563,403,410.00",9,0.29,0.27,-0.02,-538.16%,"-548,125,916.29",-542.08%,3.09,-91.19%,2021-09-02
8,solana,sol,Solana,23.13,"9,391,870,633.00",10,24.92,22.99,-1.65,-666.59%,"-672,448,219.60",-668.15%,259.96,-91.13%,2021-11-06
9,polkadot,dot,Polkadot,4.73,"5,980,946,332.00",12,4.96,4.71,-0.22,-447.38%,"-279,562,748.71",-446.55%,54.98,-91.43%,2021-11-04


In [6]:
print(df.dtypes)


id                                   object
symbol                               object
name                                 object
current_price                        object
market_cap                           object
market_cap_rank                       int64
high_24h                             object
low_24h                              object
price_change_24h                    float64
price_change_percentage_24h          object
market_cap_change_24h                object
market_cap_change_percentage_24h     object
ath                                  object
ath_change_percentage                object
ath_date                             object
dtype: object


In [16]:


# Datos proporcionados
data = {
    'id': ['bitcoin', 'ethereum', 'tether', 'binancecoin', 'ripple', 'usd-coin', 'dogecoin', 'cardano', 'solana', 'polkadot'],
    'high_24h': ['29408.00', '4000.00', '1.005', '236.00', '0.612', '1.001', '0.072', '0.281', '23.960', '4.840'],
    'low_24h': ['29053.00', '3800.00', '0.995', '235.00', '0.608', '0.999', '0.068', '0.278', '23.950', '4.830']
}

# Crea un DataFrame a partir de los datos
df_average = pd.DataFrame(data)

# Convertir columnas 'high_24h' y 'low_24h' a tipo numérico
for key in ['high_24h', 'low_24h']:
    df_average[key] = pd.to_numeric(df_average[key])

# Calcular el promedio y agregar la columna 'average_high_low'
df_average['average_high_low'] = (df_average['high_24h'] + df_average['low_24h']) / 2

# Imprimir la tabla
df_average




,id,high_24h,low_24h,average_high_low
0,bitcoin,29408.000,29053.000,29230.5000
1,ethereum,4000.000,3800.000,3900.0000
2,tether,1.005,0.995,1.0000
3,binancecoin,236.000,235.000,235.5000
4,ripple,0.612,0.608,0.6100
5,usd-coin,1.001,0.999,1.0000
6,dogecoin,0.072,0.068,0.0700
7,cardano,0.281,0.278,0.2795
8,solana,23.960,23.950,23.9550
9,polkadot,4.840,4.830,4.8350


In [17]:
df

,id,symbol,name,current_price,market_cap,market_cap_rank,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,ath,ath_change_percentage,ath_date
0,bitcoin,btc,Bitcoin,"29,079.00","566,002,645,211.00",1,"29,408.00","29,053.00",-254.49,-86.76%,"-4,949,362,744.32",-86.69%,"69,045.00",-57.87%,2021-11-10
1,ethereum,eth,Ethereum,"1,821.40","218,868,994,489.00",2,"1,842.75","1,818.53",-16.86,-91.73%,"-2,053,000,983.60",-92.93%,"4,878.26",-62.65%,2021-11-10
2,tether,usdt,Tether,1.00,"83,135,475,584.00",3,1.00,1.00,0.00,26.67%,"-208,350,642.64",-25.00%,1.32,-24.40%,2018-07-24
3,binancecoin,bnb,BNB,233.03,"35,855,531,599.00",4,239.20,232.73,-5.55,-232.76%,"-875,347,235.56",-238.31%,686.31,-66.06%,2021-05-10
4,ripple,xrp,XRP,0.59,"31,317,407,561.00",5,0.63,0.59,-0.03,-495.79%,"-1,652,308,724.09",-501.16%,3.40,-82.57%,2018-01-07
5,usd-coin,usdc,USD Coin,1.00,"26,016,993,917.00",6,1.00,1.00,0.00,11.30%,"74,236,489.00",28.62%,1.17,-14.62%,2019-05-08
6,dogecoin,doge,Dogecoin,0.07,"9,725,167,619.00",8,0.07,0.07,-0.00,-614.68%,"-629,467,308.19",-607.91%,0.73,-90.57%,2021-05-08
7,cardano,ada,Cardano,0.27,"9,563,403,410.00",9,0.29,0.27,-0.02,-538.16%,"-548,125,916.29",-542.08%,3.09,-91.19%,2021-09-02
8,solana,sol,Solana,23.13,"9,391,870,633.00",10,24.92,22.99,-1.65,-666.59%,"-672,448,219.60",-668.15%,259.96,-91.13%,2021-11-06
9,polkadot,dot,Polkadot,4.73,"5,980,946,332.00",12,4.96,4.71,-0.22,-447.38%,"-279,562,748.71",-446.55%,54.98,-91.43%,2021-11-04


In [18]:
df_concatenated = pd.concat([df, df_average['average_high_low']], axis=1)


In [19]:
df_concatenated

,id,symbol,name,current_price,market_cap,market_cap_rank,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,ath,ath_change_percentage,ath_date,average_high_low
0,bitcoin,btc,Bitcoin,"29,079.00","566,002,645,211.00",1,"29,408.00","29,053.00",-254.49,-86.76%,"-4,949,362,744.32",-86.69%,"69,045.00",-57.87%,2021-11-10,29230.5000
1,ethereum,eth,Ethereum,"1,821.40","218,868,994,489.00",2,"1,842.75","1,818.53",-16.86,-91.73%,"-2,053,000,983.60",-92.93%,"4,878.26",-62.65%,2021-11-10,3900.0000
2,tether,usdt,Tether,1.00,"83,135,475,584.00",3,1.00,1.00,0.00,26.67%,"-208,350,642.64",-25.00%,1.32,-24.40%,2018-07-24,1.0000
3,binancecoin,bnb,BNB,233.03,"35,855,531,599.00",4,239.20,232.73,-5.55,-232.76%,"-875,347,235.56",-238.31%,686.31,-66.06%,2021-05-10,235.5000
4,ripple,xrp,XRP,0.59,"31,317,407,561.00",5,0.63,0.59,-0.03,-495.79%,"-1,652,308,724.09",-501.16%,3.40,-82.57%,2018-01-07,0.6100
5,usd-coin,usdc,USD Coin,1.00,"26,016,993,917.00",6,1.00,1.00,0.00,11.30%,"74,236,489.00",28.62%,1.17,-14.62%,2019-05-08,1.0000
6,dogecoin,doge,Dogecoin,0.07,"9,725,167,619.00",8,0.07,0.07,-0.00,-614.68%,"-629,467,308.19",-607.91%,0.73,-90.57%,2021-05-08,0.0700
7,cardano,ada,Cardano,0.27,"9,563,403,410.00",9,0.29,0.27,-0.02,-538.16%,"-548,125,916.29",-542.08%,3.09,-91.19%,2021-09-02,0.2795
8,solana,sol,Solana,23.13,"9,391,870,633.00",10,24.92,22.99,-1.65,-666.59%,"-672,448,219.60",-668.15%,259.96,-91.13%,2021-11-06,23.9550
9,polkadot,dot,Polkadot,4.73,"5,980,946,332.00",12,4.96,4.71,-0.22,-447.38%,"-279,562,748.71",-446.55%,54.98,-91.43%,2021-11-04,4.8350
